The goal is to implement Bloom filter.

We do not care much for the memory efficiency here, i.e. you do not have to compactify every 8 bit values into a single machine byte. We suggest using `numpy` boolean arrays. However, bonus points will be awarded if this is done in some way (one can use `numpy.packbits`, `bitarray`, `bitstrings` or something similar, or do it manually).

The important part is choosing the source of uniform hash functions. Python has built-in `hash()` function that calls the arguments's `__hash__()` method and returns an integer which is truncated to the bit width of the host machine. One easy way to get more hash functions (distinct from a given one) is to add some salt: for example, given a string `key`, use `hash(key + salt)` instead of just `hash(key)` for some fixed `salt` string. Bonus points will be awarded if you solution works with any hashable type and not just strings.

One possible caveat is that we want a hash function of length $m$. If $m=2^d$, then a truncation of any long hash to its $d$ first (or last, or any) bits provides a hash of length $m$. If, however, $m$ is not a power of $2$, things get more complicated. In theory, considering the hash value modulo $m$ might be non-uniform, but for $m$ not too large this non-uniformity is negligible in practice (and we do want $m$ not very large, because otherwise what is the sense of using Bloom filters?). Bonus points will be awarded for an implementation of uniform hashed of any (positive) integer length (provided one explains why their solution is uniform).

The basic implementation gives 1 point, plus at most 3 bonus points, plus 1 point for the task at the end, thus this assignment can give you up to 5 points. 

In [8]:
# import numpy as np
from collections.abc import Hashable # warning: might work incorrectly for user-defined classes if their __hash__ method raises an exception
from uuid import uuid4 # generates random UUID
from bitarray import bitarray
import hashlib, pickle

In [20]:
class BloomFilter:
    def __init__(self, k: int, m: int) -> None:
        self.hash_functions_number = k
        self.hash_values = m
        self.bits = bitarray(m)
        self.bits.setall(0)

    def add(self, x: Hashable) -> None:
        for i in range(self.hash_functions_number):
            self.bits[self.hash(x, i)] = 1

    def __contains__(self, x: Hashable) -> bool:
        for i in range(self.hash_functions_number):
            if self.bits[self.hash(x, i)] == 0:
                return False
        return True

    # we could use just simple hash() function for simplicity, but this is more secure, coz hash() is not uniform between different runs
    def hash(self, value, i: int):
        return int.from_bytes(
                hashlib.sha256(pickle.dumps((i, value), protocol=pickle.HIGHEST_PROTOCOL)).digest(),
                'big'
            ) % self.hash_values

In [25]:
# Testing that the hash function works correctly
bf = BloomFilter(k=10, m=100)
for element in ['Miku', 39, (3, 9)]:
    for i in range(4):
        print(bf.hash(element, i))
    print()


84
93
15
40

28
62
87
96

74
83
17
42



Let's create a Bloom filter with some parameters $k$ and $m$ and populate it with some random UUIDs. Let's also keep track of what we added in a database.

In [34]:
db = set()
bf = BloomFilter(k=10, m=100_000)

for _ in range(10000):
    key = str(uuid4())
    db.add(key)
    bf.add(key)

We should test whether our implementation gives some false negatives.

In [35]:
false_negative_count = 0

for key in db:
    if key not in bf:
        false_negative_count += 1

false_negative_count # should be 0

0

Now let us estimate the false positives rates.

In [36]:
num_attempts = 10**5
false_positive_count = 0

for attempt in range(num_attempts):
    key = str(uuid4())
    if key in bf and key not in db:
        false_positive_count += 1

fpr = false_positive_count / num_attempts
print(fpr)

c = 0
for i in range(bf.hash_values):
    c += bf.bits[i]

print(c, bf.hash_values)

0.053
63132 100000


Does FRP align well with what the theory predicted? Run the above code for distinct values of $k$, $m$ and the number of added entries and describe your findings below.